In [7]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import time
import requests
import sys
# import base64, os

In [10]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


In [ ]:
book_name = 'release that witch'
book_name = input()

In [ ]:
try:
    seperator = '-'
    book_name = seperator.join(book_name.split(' '))
    url = f"https://boxnovel.com/novel/{book_name}/"
    browser.visit(url)
    if requests.get(url).status_code == 404:
#         sys.exit()
    time.sleep(1)
except:
    print('Welp me I broke')
#     sys.exit()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
container = soup.find_all("div", {"class": "summary-content"})
author = container[3].find("div", {"class": "author-content"}).find_all('a')

In [ ]:
with open(f'{book}.txt', 'w',encoding ='utf8') as out:
        chapter = 1
        seperator = ', '
        author_text = seperator.join([author[0].text, author[1].text])
        BookName = ' '.join([word.capitalize() for word in book.split('-')])
        out.write('\t\t\t\t\t\t\t\t\t\t\t\t'+BookName+'\n\n')
        out.write('\t\t\t\t\t\t\t\t\t\t\t'+ Author:'+author_text+'\n\n\n')
        
        try:
            url = f"https://boxnovel.com/novel/{book}/chapter-{chapter}"
            chapter +=1
            browser.visit(url)
            time.sleep(5)
            while True:
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                try:
                    container = soup.find_all("div", {"class": "read-container"})
                    text_left = container[0].find_all("div", {"class": "text-left"})
                    chp_title = text_left[0].find_all("p")[0].text
                    pgraphs = text_left[0].find_all("p")
                except:
                    print('swomethwing bwroke mwe')
                    break
                pgraphs.pop(0)
                chapter_text = [p.text for p in pgraphs]

                out.write('-----------------------------------------'+chp_title+'-----------------------------------------\n\n')
                for text in chapter_text:
                    out.write(text+'\n\n')
                try :
                    next_page = browser.driver.find_element_by_class_name('next_page')
                    next_page.click()
                except :
                    break
            except KeyboardInterrupt:
                print ('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
                try:
                    response = input()
                    if response == 'quit':
                        break
                    print ('Resuming...')
                except KeyboardInterrupt:
                    print ('Resuming...')
                    continue
    browser.driver.close()

In [130]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
book = 'Emperor of Solo Play'
# book_name = input()

try:
    seperator = '-'
    book = seperator.join(book.split(' '))
    url = f"https://www.wuxiaworld.co/{book}/"
    browser.visit(url)
#     if requests.get(url).status_code == 404:
#         browser.driver.close()
#         sys.exit()
    time.sleep(10)
except:
    print('Hwelp mwe! I bwroke.')
    try:
        browser.driver.close()
    except:
        print('nothing')
#         sys.exit()
#     sys.exit()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
container = soup.find_all("div", {"class": "person-info"})
author = container[1].find("div", {"class": "author"}).find_all('span')[1].text
chapter_list = soup.find_all("ul", {"class": "chapter-list"})
chapter_1 = browser.driver.find_element_by_class_name("chapter-item")

# In[ ]:

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


In [131]:
with open(f'{book}.txt', 'w',encoding ='utf8') as out:
    seperator = ', '
    BookName = ' '.join([word.capitalize() for word in book.split('-')])
    out.write('\t\t\t\t\t\t\t\t\t\t\t\t'+BookName+'\n\n')
    out.write('\t\t\t\t\t\t\t\t\t\t\t Author:'+author+'\n\n\n')

    try:
        # url = f"https://boxnovel.com/novel/{book}/chapter-{chapter}"
        chapter_1.click()
        # browser.visit(url)
        time.sleep(5)
        while True:
            try:
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(5)
                try:
                    container = soup.find_all("div", {"class": "chapter-entity"})
                    title = soup.find_all("h1", {"class": "chapter-title"})[0].text
                    for br in container[0].find_all("br"):
                        br.replace_with("\n")
                    chapter_text =re.sub('((\\t)*(\\n)*[\w ]* https:[/\w\.\- ]*)', '\n', container[0].text)
                    break
                except:
                    print('swomethwing bwroke mwe')
                    break
                break
                out.write('-----------------------------------------'+title+'-----------------------------------------\n\n')
                out.write(chapter_text)
                try :
                    next_page = browser.driver.find_element_by_class_name('next')
                    browser.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_page)
                    next_page.click()
                except :
                    break
            except KeyboardInterrupt:
                print ('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
                try:
                    response = input()
                    if response == 'quit':
                        break
                    print ('Resuming...')
                except KeyboardInterrupt:
                    print ('Resuming...')
                    continue
    except:
        print('Fwalure')
browser.driver.close()

In [132]:
chapter_text

'\n                        Prologue\n\nI’ve been called trash my whole life.\n\nI wasn’t born with any special talent, nor was I better at something than others. So I couldn’t say anything whenever I was called trash. That isn’t to say I never struggled to become better. But at the end of the day, I fell asleep kicking the bed sheets in anger.\n\nThis goddamned life.\n\nBut for the first time, I found something I was better at than others.\n\nVirtual reality (VR) games!\n\nIn real life, I was at the bottom of the food chain. But in the virtual world, I was at the top. Even the so called professional-gamers were mere preys in my eyes.\n\nTo top it off, you could gain unimaginable wealth and prestige just by being good at gaming. This was the era of VR games.\n\nI had no hesitation.\n\nTo succeed through gaming, I was prepared to give my life. I did anything to accomplish my goals. Everything was fair in the name of success.\n\nIn the end, the goal I’ve been striving for was within my re